In [1]:
!pip install transformers accelerate datasets torch peft pillow bitsandbytes trl

In [29]:
import os
import gc
import torch
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:128"

from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments
)
from datasets import load_dataset, DatasetDict
from peft import (
    LoraConfig,
    TaskType,
    PeftConfig,
    PeftModel,
    get_peft_model
)
from trl import SFTTrainer
from huggingface_hub import notebook_login

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = False
)

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct",
    device_map = "auto",
    quantization_config = bnb_config,
    low_cpu_mem_usage=True,
    trust_remote_code = True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

In [6]:
ds = load_dataset("aictsharif/persian-med-qa")
ds

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 209384
    })
})

In [8]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct", trust_remote_code = True)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": "[PAD]"})
    model.resize_token_embeddings(len(tokenizer))

tokenizer.truncation_side = "right"
tokenizer.model_max_length = 256

def preprocess(sample, max_length=256):
    prompt = "سوال: " + sample["question"].strip() + "\nجواب:"
    answer = " " + sample["answer"].strip()

    prompt_ids = tokenizer(prompt, add_special_tokens=False)["input_ids"]
    answer_ids = tokenizer(answer, add_special_tokens=True)["input_ids"]

    input_ids = prompt_ids + answer_ids
    if len(input_ids) > max_length:
        input_ids = input_ids[-max_length:]

    attention_mask = [1] * len(input_ids)

    pad_len = max_length - len(input_ids)
    if pad_len > 0:
        input_ids = input_ids + [tokenizer.pad_token_id] * pad_len
        attention_mask = attention_mask + [0] * pad_len

    labels = [-100] * len(prompt_ids) + answer_ids
    if len(labels) > max_length:
        labels = labels[-max_length:]
    if pad_len > 0:
        labels = labels + [-100] * pad_len

    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

In [9]:
ds_small = DatasetDict({
    "train": ds["train"].select(range(3000))
})
ds_small

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 3000
    })
})

In [10]:
data = ds_small.map(lambda s: preprocess(s, max_length=256), remove_columns=ds_small["train"].column_names)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [11]:
lora_config = LoraConfig(
    r = 16,
    lora_alpha = 16,
    target_modules = ["q_proj", "k_proj", "v_proj"],
    task_type = TaskType.CAUSAL_LM,
    lora_dropout = 0.1
)

In [12]:
def get_res(query):
    inputs = tokenizer(query, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=64,
            do_sample=False,
            num_beams=4,
            early_stopping=True,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )
    
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [13]:
print(get_res("سوال: علائم آنفولانزا چیست؟\nجواب:"))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


سوال: علائم آنفولانزا چیست؟
جواب: آنفولانزا علائم آنفولانزا علائم آنفولانزا علائم آنفولانزا علائم آنفولانزا علائم آنفولانزا علائم آنفولانزا علائم آ


In [14]:
training_args = TrainingArguments(
    num_train_epochs=5,
    learning_rate=5e-4,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    logging_steps=20,
    save_strategy="epoch",
    report_to="tensorboard",
    output_dir="/kaggle/working/",
    save_total_limit=3,
    fp16=True,
    bf16=False
)

In [15]:
trainer = SFTTrainer(
    model = model,
    peft_config = lora_config,
    train_dataset = data["train"],
    args = training_args
)

Truncating train dataset:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [16]:
gc.collect()
torch.cuda.empty_cache()

In [17]:
print("Start training...")
trainer.train()
print("Training finished...")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Start training...


Step,Training Loss
20,1.194000
40,1.006500
60,0.951800
80,0.897000
100,0.729900
120,0.751800
140,0.724900
160,0.721300
180,0.631100
200,0.598100


Training finished...


In [18]:
path = "/kaggle/working/"

In [19]:
trainer.save_model(path)
tokenizer.save_pretrained(path)

('/kaggle/working/tokenizer_config.json',
 '/kaggle/working/special_tokens_map.json',
 '/kaggle/working/chat_template.jinja',
 '/kaggle/working/vocab.json',
 '/kaggle/working/merges.txt',
 '/kaggle/working/added_tokens.json',
 '/kaggle/working/tokenizer.json')

In [20]:
config = PeftConfig.from_pretrained(path)
base = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, trust_remote_code = True)
model = PeftModel.from_pretrained(base, path)
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code = True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
def get_res(query):
    inputs = tokenizer(query, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=False,
            num_beams=4,
            early_stopping=True,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )
    
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [22]:
print(get_res("علائم کم‌خونی چیست؟"))

علائم کم‌خونی چیست؟

کم‌خونی می‌تواند علائم مختلفی داشته باشد، از جمله خستگی، رنگ پریدگی، تنگی نفس، و احساس ناامیدی. اگر شما این علائم را تجربه می‌کنید، بهتر است به پزشکی مراجعه کنید. 

خستگی و تنگی نفس معمولاً اولیه‌ای از کم‌خونی هستند،


In [23]:
print(get_res("سوال: علائم آنفولانزا چیست؟\nجواب:"))

سوال: علائم آنفولانزا چیست؟
جواب: علائم آنفولانزا شامل تب، سرفه، گلودنی چاقی، خستگی و درد مفاصل است. این علامت‌ها ممکن است بسیار شدید باشد. این بیماری معمولاً ۲ تا ۵ روز طول می‌کند. اگر علائم شدید یا استرس‌آور باشد، می‌تواند نیاز به مشاوره‌های پز


In [25]:
print(get_res("چگونه فشار خون را کنترل کنیم؟"))

چگونه فشار خون را کنترل کنیم؟

کنترل فشار خون شامل تغییرات در رژیم غذایی، ورزش، کنترل استرس و در برخی موارد دارو است. برای کنترل فشار خون، می‌توان از رژیم غذایی سالم، ورزش منظم، کنترل استرس و در برخی موارد دارو استفاده کرد. 

1. رژیم غذایی سالم: می‌توان از غ


In [26]:
print(get_res("چگونه می‌توان چربی خون را کاهش داد؟"))

چگونه می‌توان چربی خون را کاهش داد؟

چربی خون یک بیماری مزمن است که باعث ایجاد چربی در سیستم اندام‌ها می‌شود. کاهش چربی خون می‌تواند با تغییرات در رژیم غذایی، فعالیت بدنی و در برخی موارد دارو امکان‌پذیر باشد. برای کاهش چربی خون، می‌توان از غذاهای


In [27]:
def get_res(query):
    inputs = tokenizer(query, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=256,              
            do_sample=True,                  
            top_p=0.9,
            temperature=0.7,
            num_beams=4,                     
            early_stopping=False,            
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )
    
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [28]:
print(get_res("چگونه می‌توان چربی خون را کاهش داد؟"))

چگونه می‌توان چربی خون را کاهش داد؟

چربی خون یک بیماری مزمن است که باعث ایجاد چربی در سیستم اندام‌ها می‌شود. کاهش چربی خون می‌تواند با تغییرات در رژیم غذایی، فعالیت بدنی و در برخی موارد دارو امکان‌پذیر باشد. برای کاهش چربی خون، پیشنهاد می‌شود:

1. تغییر رژیم غذایی:
- کاهش ترکیبی از الکل و دیالکتیک
- رژیم غذایی کم‌چرب
- مصرف میوه و سبزیجات

2. افزایش فعالیت بدنی:
- ورزش منظم
- کنترل وزن

3. مدیریت استرس:
- تکنیک‌های خواب کافی
- تکنیک‌


In [30]:
notebook_login()

In [31]:
model.push_to_hub("alikhademi98/medical_question_answerig_Qwen2.5-3B-Instruct_qlora")
tokenizer.push_to_hub("alikhademi98/medical_question_answerig_Qwen2.5-3B-Instruct_qlora")

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/alikhademi98/medical_question_answerig_Qwen2.5-3B-Instruct_qlora/commit/492313d5f98455532cd3bca16006d2d5543f09c1', commit_message='Upload tokenizer', commit_description='', oid='492313d5f98455532cd3bca16006d2d5543f09c1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/alikhademi98/medical_question_answerig_Qwen2.5-3B-Instruct_qlora', endpoint='https://huggingface.co', repo_type='model', repo_id='alikhademi98/medical_question_answerig_Qwen2.5-3B-Instruct_qlora'), pr_revision=None, pr_num=None)